In [ ]:
# turn autoreload on for Jupyter notebook 
%load_ext autoreload
%autoreload 2

import string
from prettytable import *

import cPickle as pickle

In [ ]:
from voting.voter import *
from voting.profile import *
from voting.voting_methods import *

## Requirements
1. There is a directory called output with a subdirectory called heatmaps
2. The directory voting_data has a file called voting_scenarios3.pkl.   This pickle file was created by the CreateReducedProfiles notebook (the pickle file needs to be moved to the voting_data directory after being created)

In [ ]:
voting_scenarios = pickle.load(open("./voting_data/voting_scenarios3.pkl", "rb"))

In [ ]:

voting_sizes = [(3,4), (3,5), (3,6), (3,7), (3,8), (4,3), (4,4)]

set_preferences = [("AAdom", "weak_dom"), 
                   ("weak_optimist_prefers", "optimist_prefers"), 
                   ("weak_pessimist_prefers", "pessimist_prefers")]


voting_methods =   [baldwin,
                    borda, 
                    condorcet,
                    coombs,    
                    copeland, 
                    hare,
                    maxmin,
                    plurality, 
                    plurality_with_runoff,                    
                    strict_nanson,
                    weak_nanson]

max_vms_set_size = 3 #len(voting_methods)

sure_manipulations = {vsize: dict() for vsize in voting_sizes}
safe_manipulations = {vsize: dict() for vsize in voting_sizes}
expected_manipulations = {vsize: dict() for vsize in voting_sizes}
stochastic_manipulations = {vsize: dict() for vsize in voting_sizes}


In [ ]:
%%time

from functools import partial
from multiprocessing import Pool
from multiprocessing import cpu_count

def find_manipulations(voting_methods, pointed_profile, set_preference, rankings):
    
    # return the manipulations of a pointed profile for a set of voting methods and set preference
    # if set_prference is None, only  find stochastic dominance
    
    manips = {
        "sure_manips": list(),
        "safe_manips": list(),
        "expected_manips": list(),
        "stochastic_manips": list()
    }
    
    voter = pointed_profile.voter
    
    if set_preference is not None:
        # get the strict set preference for the voter
        v_actual_ranking_set_preference = getattr(voter, set_preference[1])

        # get the weak set preference for the voter
        v_actual_ranking_weak_set_preference = getattr(voter, set_preference[0])
    
    # winning sets for acutal  profile
    pprof_winning_sets = [vm(pointed_profile) for vm in voting_methods]
   
    for new_ranking in rankings:
        
        # create new profile from new_ranking
        updated_pointed_profile = pointed_profile.create_new_profile(new_ranking)
        
        # new winning sets
        updated_pprof_winning_sets = [vm(updated_pointed_profile) for vm in voting_methods]

        # find manipulation instances for dominance notions
        
        if set_preference is not None and voter.sure_dominance(updated_pprof_winning_sets,
                                                               pprof_winning_sets,
                                                               v_actual_ranking_set_preference):            
            manips["sure_manips"].append(new_ranking)
        
        if set_preference is not None and voter.safe_dominance(updated_pprof_winning_sets,
                                                               pprof_winning_sets,
                                                               v_actual_ranking_weak_set_preference,
                                                               v_actual_ranking_set_preference):            
            manips["safe_manips"].append(new_ranking)

        if set_preference is not None and voter.expected_dominance(updated_pprof_winning_sets, 
                                                                   pprof_winning_sets,
                                                                   v_actual_ranking_set_preference):           
            manips["expected_manips"].append(new_ranking)

        if set_preference is None and voter.stochastic_dominance(updated_pprof_winning_sets,
                                                                 pprof_winning_sets, 
                                                                 pointed_profile.candidates):            
            manips["stochastic_manips"].append(new_ranking)

    return manips


def get_strategizing_instances(voter_names, 
                               voting_methods, 
                               all_rankings, 
                               max_vms_set_size, 
                               set_preferences, 
                               _prof):
    
    voting_methods_sets = [vms for vms_size in range(1, max_vms_set_size + 1) 
                           for vms in create_subsets(voting_methods, vms_size)]
    
    sure_manip_instances = {dom_to_str(sp):{vms_to_str(vms):{"num_pointed_profiles": 0,
                                                             "num_profiles": 0,
                                                             "instances": list()}
                                            for vms in voting_methods_sets}
                            for sp in set_preferences}
    
    safe_manip_instances = {dom_to_str(sp):{vms_to_str(vms):{"num_pointed_profiles": 0,
                                                             "num_profiles": 0,
                                                             "instances": list()}
                                            for vms in voting_methods_sets}
                            for sp in set_preferences}
    
    expected_manip_instances = {dom_to_str(sp):{vms_to_str(vms):{"num_pointed_profiles": 0,
                                                                 "num_profiles": 0,
                                                                 "instances": list()}
                                                for vms in voting_methods_sets}
                                for sp in set_preferences}
    
    stochastic_manip_instances = {vms_to_str(vms):{"num_pointed_profiles": 0,
                                                "num_profiles": 0,
                                                "instances": list()}
                                  for vms in voting_methods_sets}
     
    for vms in voting_methods_sets:
        
        stochastic_manipulations = {vname:list() for vname in voter_names}
        
        for vname in voter_names: 

            pprof = create_pointed_profile(voter_names, _prof, vname)
            manips = find_manipulations(vms, pprof, None, all_rankings)
            
            stochastic_manipulations[vname] = manips["stochastic_manips"]
         
        
        manipulations = stochastic_manipulations
        if any([len(manipulations[_vn]) > 0 for _vn in voter_names]):
            stochastic_manip_instances[vms_to_str(vms)]["num_pointed_profiles"] += len([_vn for _vn in voter_names 
                                                                                        if len(manipulations[_vn]) > 0])
            stochastic_manip_instances[vms_to_str(vms)]["num_profiles"] += 1
            stochastic_manip_instances[vms_to_str(vms)]["instances"].append({"profile":pprof.as_dict(),
                                                                             "manipulations": manipulations})

        for sp in set_preferences: 

            sure_manipulations = {vname:list() for vname in voter_names}
            safe_manipulations = {vname:list() for vname in voter_names}
            expected_manipulations = {vname:list() for vname in voter_names}

            for vname in voter_names: 

                pprof = create_pointed_profile(voter_names, _prof, vname)
                manips = find_manipulations(vms, pprof, sp, all_rankings)

                sure_manipulations[vname] = manips["sure_manips"]
                safe_manipulations[vname] = manips["safe_manips"]
                expected_manipulations[vname] = manips["expected_manips"]

            s_instances = sure_manip_instances[dom_to_str(sp)][vms_to_str(vms)]
            manipulations = sure_manipulations
            if any([len(manipulations[_vn]) > 0 for _vn in voter_names]):
                s_instances["num_pointed_profiles"] += len([_vn for _vn in voter_names 
                                                            if len(manipulations[_vn]) > 0])
                s_instances["num_profiles"] += 1
                s_instances["instances"].append({"profile":pprof.as_dict(),
                                                 "manipulations": manipulations})

            s_instances = safe_manip_instances[dom_to_str(sp)][vms_to_str(vms)]
            manipulations = safe_manipulations
            if any([len(manipulations[_vn]) > 0 for _vn in voter_names]):
                s_instances["num_pointed_profiles"] += len([_vn for _vn in voter_names 
                                                            if len(manipulations[_vn]) > 0])
                s_instances["num_profiles"] += 1
                s_instances["instances"].append({"profile":pprof.as_dict(),
                                                 "manipulations": manipulations})

            s_instances = expected_manip_instances[dom_to_str(sp)][vms_to_str(vms)]
            manipulations = expected_manipulations
            if any([len(manipulations[_vn]) > 0 for _vn in voter_names]):
                s_instances["num_pointed_profiles"] += len([_vn for _vn in voter_names 
                                                            if len(manipulations[_vn]) > 0])
                s_instances["num_profiles"] += 1
                s_instances["instances"].append({"profile":pprof.as_dict(),
                                                 "manipulations": manipulations})

    
    return {"sure": sure_manip_instances, 
            "safe": safe_manip_instances, 
            "expected": expected_manip_instances, 
            "stochastic": stochastic_manip_instances}




In [ ]:
import seaborn as sns
import pandas as pd
from itertools import product
import numpy as np
import matplotlib.pyplot as plt

def get_manipulation_pointed_profile_precentage(manipulations,  voting_methods, voting_size, dominance):
    
    total_pointed_profiles = manipulations["total_pointed_profiles"]
    
    if dominance is None: 
        # it is stochastic dominance
        
        vms_key = filter(lambda vms_str: same_voting_methods_string(vms_str, vms_to_str(voting_methods)),
                         manipulations["manip_instances"].keys())[0]
        
        num_manip_pointed_profiles = manipulations["manip_instances"][vms_key]["num_pointed_profiles"]
        
    else:
        # it is one of the others...
        vms_key = filter(lambda vms_str: same_voting_methods_string(vms_str, vms_to_str(voting_methods)),
                         manipulations["manip_instances"][dom_to_str(dominance)].keys())[0]
        
        num_manip_pointed_profiles = manipulations["manip_instances"][dom_to_str(dominance)][vms_key]["num_pointed_profiles"]


    return round(float(num_manip_pointed_profiles) / float(total_pointed_profiles), 5) * 100


def get_manipulation_profile_precentage(manipulations, voting_methods, voting_size, dominance):
    
    total_pointed_profiles = manipulations["total_profiles"]
    
    if dominance is None: 
        # it is stochastic dominance
        
        vms_key = filter(lambda vms_str: same_voting_methods_string(vms_str, vms_to_str(voting_methods)),
                         manipulations["manip_instances"].keys())[0]
        
        num_manip_pointed_profiles = manipulations["manip_instances"][vms_key]["num_profiles"]
        
    else:
        # it is one of the others...
        vms_key = filter(lambda vms_str: same_voting_methods_string(vms_str, vms_to_str(voting_methods)),
                         manipulations["manip_instances"][dom_to_str(dominance)].keys())[0]
        
        num_manip_pointed_profiles = manipulations["manip_instances"][dom_to_str(dominance)][vms_key]["num_profiles"]


    return round(float(num_manip_pointed_profiles) / float(total_pointed_profiles), 5) * 100


def create_heat_map(manip_instances, manip_type, profile_type, dom, num_cands, num_voters): 

    prof_precentage_fnc = get_manipulation_pointed_profile_precentage if profile_type  == "pointed profiles" else get_manipulation_profile_precentage

    voting_methods =  all_voting_methods = [borda, 
                                            strict_nanson, 
                                            weak_nanson,
                                            maxmin,
                                            hare,
                                            coombs,                                         
                                            baldwin,
                                            condorcet, 
                                            copeland, 
                                            plurality, 
                                            plurality_with_runoff]

    vm_str_to_vm = lambda s: {vms_to_str(vms): list(vms) for vms in create_subsets(voting_methods,1) + create_subsets(voting_methods, 2)}[s]

    num_voting_methods = len(voting_methods)


    vm_names = [vms_to_str(vms) for vms in create_subsets(voting_methods, 1)]
    vm_names_added = vm_names #list(vm_str_to_vm.keys())

    vm_names_prod = list(product(vm_names,vm_names_added))

    def modify_vm_name(vm): 
        if vm == "Nanson":
            return "StrictNanson"
        elif vm == "Nanson Weak":
            return "WeakNanson"
        else:
            return vm 

    d = {idx: {"voting method": modify_vm_name(v[0]), 
               "voting method ": modify_vm_name(v[1]), 
               "manipnum": prof_precentage_fnc(manip_instances,
                                               vm_str_to_vm(v[0]) + vm_str_to_vm(v[1]),
                                               (num_cands,num_voters),
                                               dom)} 
         for idx, v in enumerate(vm_names_prod)}
    
    df = None
    ax = None
    df = pd.DataFrame(d)
    df= df.transpose()
    #df
    #get_num_manip(sure_manipulation, (3,4), voting_method_strings, ("AAdom", "strong_dom"))
    #print df
    df["manipnum"] = pd.to_numeric(df["manipnum"])
    df = df.pivot("voting method ", "voting method", "manipnum")

    plt.figure(figsize=(11,6))
    #print df
    sns.set(rc={'figure.figsize':(11,6)})
    corr = np.corrcoef(np.random.randn(num_voting_methods, num_voting_methods))
    mask = np.zeros_like(corr)
    #print np.triu_indices_from(mask, k=100)
    mask[np.triu_indices_from(mask, k=1)] = True

    cmap=plt.get_cmap("Greens")

    if manip_type in ["sure", "safe", "expected"]: 
        heat_map_filename = "{}_manipulations_{}_{}_{}.pdf".format(manip_type.replace(" ", "_"),
                                                                   num_cands, 
                                                                   num_voters,
                                                                   profile_type.replace(" ", "_"))
    else:
        heat_map_filename = "{}_manipulations_{}_{}_{}.pdf".format(manip_type.replace(" ", "_"), 
                                                                   num_cands, 
                                                                   num_voters,
                                                                   profile_type.replace(" ", "_"))


    with sns.axes_style("white"):
        ax = sns.heatmap(df, annot=True, mask=mask, linewidth=1,cmap=cmap)
        ax.set_title('Percentage of {} witnessesing {} manipulation for ({}, {})\n'.format(profile_type, 
                                                                                           manip_type, 
                                                                                           num_cands, 
                                                                                           num_voters))
        fig = ax.get_figure()

        fig.savefig("./output/heatmaps/" + heat_map_filename, format="pdf",bbox_inches="tight", pad_inches=0.5)


In [ ]:
%%time

import json

voting_sizes = [(3,4), (3,5), (3,6), (3,7)]

for vsize in voting_sizes:
    
    
    vs =  filter(lambda vs: vs["num_candidates"] == vsize[0] and vs["num_voters"] == vsize[1], voting_scenarios)[0]

    num_cands = vs["num_candidates"]
    num_voters = vs["num_voters"]
    candidates = vs["candidates"]
    voter_names = vs["voter_names"]
    all_rankings = vs["all_orderings"]

    profiles = vs["profiles"]

    num_profiles = len(profiles)
    num_pointed_profiles = len(vs["pointed_profiles"])

    voting_methods_sets = [vms for vms_size in range(1, max_vms_set_size + 1) 
                           for vms in create_subsets(voting_methods, vms_size)]

    print
    print "{} candidates and {} voters".format(num_cands, num_voters)
    print "There are {} profiles".format(num_profiles)
    print "There are {} voting method sets".format(len(voting_methods_sets))
    print 
    print "Finding straegizing instances...."

    chunksize = int(0.05 * len( vs["profiles"]))
    print "chunksize is {}".format(chunksize)
    p = Pool(None)
    print "Using {} CPUs".format(cpu_count())

    profile_to_strat_instance = partial(get_strategizing_instances,
                                        voter_names,
                                        voting_methods,
                                        all_rankings,
                                        max_vms_set_size,
                                        set_preferences)

    strategizing_instances = p.map(profile_to_strat_instance, profiles) #, chunksize=chunksize)

    print "\nSaving output..."
    print



    sure_strat_instances = {"num_candidates": num_cands,
                            "num_voters": num_voters,
                            "candidates": candidates,
                            "voter_names": voter_names,
                            "all_rankings": all_rankings,
                            "total_profiles": num_profiles,
                            "total_pointed_profiles": num_pointed_profiles,
                            "manip_instances":{dom_to_str(sp):{vms_to_str(vms):{"num_pointed_profiles": 0,
                                                                                "num_profiles": 0,
                                                                                "instances": list()}
                                                               for vms in voting_methods_sets}
                                               for sp in set_preferences}}

    safe_strat_instances = {"num_candidates": num_cands,
                            "num_voters": num_voters,
                            "candidates": candidates,
                            "voter_names": voter_names,
                            "all_rankings": all_rankings,
                            "total_profiles": num_profiles,
                            "total_pointed_profiles": num_pointed_profiles,
                            "manip_instances":{dom_to_str(sp):{vms_to_str(vms):{"num_pointed_profiles": 0,
                                                                                "num_profiles": 0,
                                                                                "instances": list()}
                                                               for vms in voting_methods_sets}
                                               for sp in set_preferences}}

    expected_strat_instances = {"num_candidates": num_cands,
                                "num_voters": num_voters,                        
                                "candidates": candidates,
                                "voter_names": voter_names,
                                "all_rankings": all_rankings,
                                "total_profiles": num_profiles,
                                "total_pointed_profiles": num_pointed_profiles,                        
                                "manip_instances":{dom_to_str(sp):{vms_to_str(vms):{"num_pointed_profiles": 0, 
                                                                                    "num_profiles": 0,  
                                                                                    "instances": list()} 
                                                                   for vms in voting_methods_sets}                                           
                                                   for sp in set_preferences}}

    stochastic_strat_instances = {"num_candidates": num_cands,
                                  "num_voters": num_voters,                        
                                  "candidates": candidates,
                                  "voter_names": voter_names,
                                  "all_rankings": all_rankings,
                                  "total_profiles": num_profiles,
                                  "total_pointed_profiles": num_pointed_profiles,                                                    
                                  "manip_instances":{vms_to_str(vms):{"num_pointed_profiles": 0, 
                                                                      "num_profiles": 0,
                                                                      "instances": list()}
                                                     for vms in voting_methods_sets}}

    for si in strategizing_instances:
        for vms in voting_methods_sets:

            if si["stochastic"][vms_to_str(vms)]["num_profiles"] == 1:
                stochastic_strat_instances["manip_instances"][vms_to_str(vms)]["num_profiles"]+=1
                stochastic_strat_instances["manip_instances"][vms_to_str(vms)]["num_pointed_profiles"]+=si["stochastic"][vms_to_str(vms)]["num_pointed_profiles"]
                stochastic_strat_instances["manip_instances"][vms_to_str(vms)]["instances"]+=si["stochastic"][vms_to_str(vms)]["instances"]

            for sp in set_preferences:

                if si["sure"][dom_to_str(sp)][vms_to_str(vms)]["num_profiles"] == 1:
                    sure_strat_instances["manip_instances"][dom_to_str(sp)][vms_to_str(vms)]["num_profiles"]+=1
                    sure_strat_instances["manip_instances"][dom_to_str(sp)][vms_to_str(vms)]["num_pointed_profiles"]+=si["sure"][dom_to_str(sp)][vms_to_str(vms)]["num_pointed_profiles"]
                    sure_strat_instances["manip_instances"][dom_to_str(sp)][vms_to_str(vms)]["instances"]+=si["sure"][dom_to_str(sp)][vms_to_str(vms)]["instances"]

                if si["safe"][dom_to_str(sp)][vms_to_str(vms)]["num_profiles"] == 1:
                    safe_strat_instances["manip_instances"][dom_to_str(sp)][vms_to_str(vms)]["num_profiles"]+=1
                    safe_strat_instances["manip_instances"][dom_to_str(sp)][vms_to_str(vms)]["num_pointed_profiles"]+=si["safe"][dom_to_str(sp)][vms_to_str(vms)]["num_pointed_profiles"]
                    safe_strat_instances["manip_instances"][dom_to_str(sp)][vms_to_str(vms)]["instances"]+=si["safe"][dom_to_str(sp)][vms_to_str(vms)]["instances"]

                if si["expected"][dom_to_str(sp)][vms_to_str(vms)]["num_profiles"] == 1:
                    expected_strat_instances["manip_instances"][dom_to_str(sp)][vms_to_str(vms)]["num_profiles"]+=1
                    expected_strat_instances["manip_instances"][dom_to_str(sp)][vms_to_str(vms)]["num_pointed_profiles"]+=si["expected"][dom_to_str(sp)][vms_to_str(vms)]["num_pointed_profiles"]
                    expected_strat_instances["manip_instances"][dom_to_str(sp)][vms_to_str(vms)]["instances"]+=si["expected"][dom_to_str(sp)][vms_to_str(vms)]["instances"]


    #print json.dumps(sure_strat_instances, indent=4)

    with open("./output/sure_manipulations_{}cands_{}voters.json".format(num_cands, num_voters), 'w') as outfile:  
        json.dump(sure_strat_instances, outfile, indent=2)

    with open("./output/safe_manipulations_{}cands_{}voters.json".format(num_cands, num_voters), 'w') as outfile:  
        json.dump(safe_strat_instances, outfile, indent=2)

    with open("./output/expected_manipulations_{}cands_{}voters.json".format(num_cands, num_voters), 'w') as outfile:  
        json.dump(expected_strat_instances, outfile, indent=2)

    with open("./output/stochastic_manipulations_{}cands_{}voters.json".format(num_cands, num_voters), 'w') as outfile:  
        json.dump(stochastic_strat_instances, outfile, indent=2)

    sure_manipulations[(num_cands, num_voters)] = sure_strat_instances
    safe_manipulations[(num_cands, num_voters)] = safe_strat_instances
    expected_manipulations[(num_cands, num_voters)] = expected_strat_instances
    stochastic_manipulations[(num_cands, num_voters)] = stochastic_strat_instances

    print "Creating heatmaps for pointed profiles...."
    
    dominances = [(sure_manipulations, "sure weak dominance", ("AAdom", "weak_dom")),
                  (sure_manipulations, "sure pessimist dominance", ("weak_pessimist_prefers", "pessimist_prefers")),
                  (sure_manipulations, "sure optimist dominance", ("weak_optimist_prefers", "optimist_prefers")),
                  (safe_manipulations, "safe weak dominance", ("AAdom", "weak_dom")),
                  (safe_manipulations, "safe pessimist dominance", ("weak_pessimist_prefers", "pessimist_prefers")),
                  (safe_manipulations, "safe optimist dominance", ("weak_optimist_prefers", "optimist_prefers")),
                  (expected_manipulations, "expected weak dominance", ("AAdom", "weak_dom")),
                  (expected_manipulations, "expected pessimist dominance", ("weak_pessimist_prefers", "pessimist_prefers")),
                  (expected_manipulations, "expected optimist dominance", ("weak_optimist_prefers", "optimist_prefers")),      
                  (stochastic_manipulations, "stochastic dominance", None)]

    for doms in dominances:
        create_heat_map(doms[0][(num_cands, num_voters)], 
                        doms[1], 
                        "pointed profiles", 
                        doms[2], 
                        num_cands, 
                        num_voters)
        
    print "Creating heatmaps for profiles...."

    for doms in dominances:
        create_heat_map(doms[0][(num_cands, num_voters)], 
                        doms[1], 
                        "profiles", 
                        doms[2], 
                        num_cands, 
                        num_voters)
    print "Done.\n\n"

pickle.dump(sure_manipulations,open("./output/sure_manipulations.pkl","w"))
pickle.dump(safe_manipulations,open("./output/safe_manipulations.pkl","w"))
pickle.dump(expected_manipulations,open("./output/expected_manipulations.pkl","w"))
pickle.dump(stochastic_manipulations,open("./output/stochastic_manipulations.pkl","w"))
